In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from pandas_profiling import ProfileReport

In [2]:
dirty_train = pd.read_csv("../../../datasets/dataset_5/dataset_5_train_merged.csv").set_index("user_id")
dirty_test = pd.read_csv("../../../datasets/dataset_5/dataset_5_test_merged.csv").set_index("user_id")
dirty_test.drop("CHURN", axis=1, inplace=True)

In [3]:
def data_preparation (train_y):
    train_y.drop('MRG', axis=1, inplace = True)
    train_y.drop('ARPU_SEGMENT', axis=1, inplace = True)
    
    train_y['TOP_PACK'].fillna('Unknown', inplace = True)
    train_y['REGION'].fillna('UNKNOWN', inplace=True)
    train_y["REGULARITY_TOP_PACK_POPULARITY"] = train_y.groupby(["REGULARITY", "TOP_PACK"])["user_id"].transform("count")
    train_y["REGION_TOP_PACK_POPULARITY"] = train_y.groupby(["REGION", "TOP_PACK"])["user_id"].transform("count")
    
    train_y['REGION'].replace(to_replace='UNKNOWN', value=np.nan, inplace=True)
    train_y['TOP_PACK'].replace(to_replace='Unknown', value=np.nan, inplace=True)
    try:
        train_y.set_index('user_id', inplace = True)
    except:
        pass
    try:
        train_y.drop('user_id', inplace = True, axis=1)
    except:
        pass
    
    target = None
    if 'CHURN' in train_y.columns:
        target = train_y['CHURN']
        train_y.drop('CHURN', inplace=True, axis=1)
    #Feature generating

    #Bits
    # Sum calls+data
    train_y.insert(train_y.shape[1], 'TELE_NANS_SUM', train_y[['DATA_VOLUME','ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2']].isnull().sum(axis=1))
    # Sum of another
    train_y.insert(train_y.shape[1], 'NONTELE_NANS_SUM', train_y[train_y.columns.difference(['DATA_VOLUME','ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2'])].isnull().sum(axis=1))
    # Bit for REGION?
    
    train_y.insert(train_y.shape[1], 'MONTANT_TO_REVENUE', train_y['MONTANT']/train_y['REVENUE'])
    train_y['MONTANT_TO_REVENUE'].fillna(0, inplace=True)
    
    #train_y.insert(train_y.shape[1], 'FREQ_DIFF', train_y['FREQUENCE']-train_y['FREQUENCE_RECH'])
    #train_y['FREQ_DIFF'].fillna(0.0, inplace=True)
    
    train_y.insert(train_y.shape[1], 'FREQ_DIFF_RELATIVE', ((train_y['FREQUENCE']-train_y['FREQUENCE_RECH'])/train_y['FREQUENCE']).astype(np.single))
    train_y['FREQ_DIFF_RELATIVE'].fillna(999.0, inplace=True)
    
    train_y.insert(train_y.shape[1], 'FULL_CALLS_SUM', train_y[['ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2']].sum(axis=1))
    train_y['FULL_CALLS_SUM'].fillna(-1, inplace=True)

    train_y.insert(train_y.shape[1], 'ON_NET_REL', train_y['ON_NET']/(train_y['FULL_CALLS_SUM'].astype(np.single)))
    train_y['ON_NET_REL'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ORANGE_REL', train_y['ORANGE']/(train_y['FULL_CALLS_SUM'].astype(np.single)))
    train_y['ORANGE_REL'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'TIGO_REL', train_y['TIGO']/(train_y['FULL_CALLS_SUM'].astype(np.single)))
    train_y['TIGO_REL'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ZONE1_REL', train_y['ZONE1']/(train_y['FULL_CALLS_SUM'].astype(np.single)))
    train_y['ZONE1_REL'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ZONE2_REL', train_y['ZONE2']/(train_y['FULL_CALLS_SUM'].astype(np.single)))
    train_y['ZONE2_REL'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'FULL_CALLS_SUM_TO_REGULATIRY', (1.0*train_y[['ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2']].sum(axis=1))/(train_y['REGULARITY']*1.0)) #нужен тут astype или нет?
    train_y['FULL_CALLS_SUM_TO_REGULATIRY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'DATA_VOLUME_TO_REGULARITY', (train_y['DATA_VOLUME'].astype(np.single)/(train_y['REGULARITY'].astype(np.single))))
    train_y['DATA_VOLUME_TO_REGULARITY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ON_NET_TO_REGULARITY', (train_y['ON_NET']/(train_y['REGULARITY'].astype(np.single))))
    train_y['ON_NET_TO_REGULARITY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ORANGE_TO_REGULARITY', (train_y['ORANGE']/(train_y['REGULARITY'].astype(np.single))))
    train_y['ORANGE_TO_REGULARITY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'TIGO_TO_REGULARITY', (train_y['TIGO']/(train_y['REGULARITY'].astype(np.single))))
    train_y['TIGO_TO_REGULARITY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ZONE1_TO_REGULARITY', (train_y['ZONE1']/(train_y['REGULARITY'].astype(np.single))))
    train_y['ZONE1_TO_REGULARITY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'ZONE2_TO_REGULARITY', (train_y['ZONE2']/(train_y['REGULARITY'].astype(np.single))))
    train_y['ZONE2_TO_REGULARITY'].fillna(-1, inplace=True)
    
    train_y.insert(train_y.shape[1], 'FREQUENCE_TO_REGULARITY', (train_y['FREQUENCE']/(train_y['REGULARITY'].astype(np.single))))
    train_y['FREQUENCE_TO_REGULARITY'].fillna(0.0, inplace=True)
    
    train_y.insert(train_y.shape[1], 'FREQUENCE_RECH_TO_REGULARITY', (train_y['FREQUENCE_RECH']/(train_y['REGULARITY'].astype(np.single))))
    train_y['FREQUENCE_RECH_TO_REGULARITY'].fillna(0.0, inplace=True)
    
    train_y.insert(train_y.shape[1], 'REVENUE/REGULARITY', train_y['REVENUE']/train_y['REGULARITY'].astype(np.single))
    train_y['REVENUE/REGULARITY'].fillna(0.0, inplace=True)
    
    train_y['TENURE_DURATION'] = train_y['TENURE'].map({'K > 24 month': 8, 'I 18-21 month': 6, 'H 15-18 month': 5, 'G 12-15 month':4,
                                                        'J 21-24 month': 7, 'F 9-12': 3, 'E 6-9 month':2, 'D 3-6 month':1})
    
    # Categorical TENURE
    sorted_tenure = {i : j for i, j in zip(np.sort(train_y['TENURE'].unique()), range(8))}
    train_y['TENURE'].replace(to_replace=sorted_tenure, inplace=True)

    
    # Update 27.09
    # The only full nans rows info
    #train_y['TEN_REGULARITY_MEDIAN']=train_y.groupby(['TENURE'])['REGULARITY'].apply(lambda x: x-x.median())
    #train_y['TEN_REGULARITY_MEAN']=train_y.groupby(['TENURE'])['REGULARITY'].apply(lambda x: x-x.mean())
    
    # Categorical REGION
    
    sorted_region = {i : j for i, j in zip(train_y['REGION'].value_counts().index, range(len(train_y['REGION'].value_counts().index)))}
    train_y['REGION'].replace(to_replace=sorted_region, inplace=True)
    
    # Update 27.09
    #train_y['REG_TEN_REGULARITY_MEDIAN']=train_y.groupby(['REGION', 'TENURE'])['REGULARITY'].apply(lambda x: x-x.median())
    #train_y['REG_TEN_REGULARITY_MEAN']=train_y.groupby(['REGION', 'TENURE'])['REGULARITY'].apply(lambda x: x-x.mean())
    
    #tarif_dict = train_y['TOP_PACK'].value_counts().to_dict()
    
    #Manual segmentation based on names
    tarif_time = {'EVC_1000=6000 F' : 'o',
                  'EVC_PACK_2.2Go' : 'o',
                  'All-net 500F=2000F;5d' : 'd',
                  'On net 200F=Unlimited _call24H': 'h',
                  'Data:490F=1GB,7d' : 'w',
                  'Data: 100 F=40MB,24H' : 'h',
                  'Mixt 250F=Unlimited_call24H' : 'h',
                  'MIXT:500F= 2500F on net _2500F off net;2d' : 'd',
                  'Data:1000F=2GB,30d' : 'm',
                  'All-net 500F =2000F_AllNet_Unlimited' : 'o',
                  'Jokko_Daily' : 'd',
                  'Data: 200 F=100MB,24H' : 'h',
                  'IVR Echat_Daily_50F' : 'd',
                  'On-net 500=4000,10d' : 'w',
                  'On-net 500F_FNF;3d' : 'd',
                  'Data:200F=Unlimited,24H' : 'h',
                  'MIXT: 200mnoff net _unl on net _5Go;30d' : 'm',
                  'On-net 1000F=10MilF;10d' : 'w',
                  'VAS(IVR_Radio_Daily)' : 'd',
                  'Data:1000F=5GB,7d' : 'w',
                  'Twter_U2opia_Daily' : 'd',
                  'All-net 600F= 3000F ;5d' : 'd',
                  'On-net 200F=60mn;1d' : 'h',
                  'All-net 1000=5000;5d' : 'd',
                  'Data:500F=2GB,24H' : 'h',
                  'Twter_U2opia_Weekly' : 'w',
                  'Data:3000F=10GB,30d' : 'm',
                  'MIXT: 390F=04HOn-net_400SMS_400 Mo;4h\t' : 'm',
                  'All-net 500F=1250F_AllNet_1250_Onnet;48h' : 'd',
                  'Data:300F=100MB,2d' : 'd',
                  'Data:50F=30MB_24H' : 'h',
                  'All-net 1000F=(3000F On+3000F Off);5d' : 'd',
                  'On net 200F= 3000F_10Mo ;24H' : 'h',
                  'MIXT: 590F=02H_On-net_200SMS_200 Mo;24h\t\t' : 'h',
                  'Data:DailyCycle_Pilot_1.5GB' : 'd',
                  'All-net 300=600;2d' : 'd',
                  'Data:150F=SPPackage1,24H' : 'h',
                  '200=Unlimited1Day' : 'h', #?
                  'Data:1500F=3GB,30D' : 'm',
                  'Jokko_promo' : 'o',
                  'Data:700F=1.5GB,7d' : 'w',
                  'On-net 300F=1800F;3d' : 'd',
                  'Facebook_MIX_2D' : 'o',
                  'Data:30Go_V 30_Days' : 'm',
                  'Pilot_Youth4_490': 'o',
                  'CVM_on-net bundle 500=5000': 'o',
                  'MROMO_TIMWES_RENEW': 'o',
                  'Jokko_Monthly': 'm',
                  'MROMO_TIMWES_OneDAY': 'h',
                  'WIFI_Family_2MBPS': 'o',
                  'Data:1500F=SPPackage1,30d': 'm',
                  'Yewouleen_PKG': 'o',
                  'Twter_U2opia_Monthly': 'm',
                  'YMGX 100=1 hour FNF, 24H/1 month': 'm',
                  'New_YAKALMA_4_ALL': 'o',
                  'Data: 490F=Night,00H-08H': 'h',
                  'Pilot_Youth1_290': 'o',
                  'Internat: 1000F_Zone_1;24H\t\t': 'h',
                  'All-net 5000= 20000off+20000on;30d': 'm',
                  'SUPERMAGIK_5000': 'o',
                  'On-net 2000f_One_Month_100H; 30d': 'm',
                  'MIXT:1000F=4250 Off net _ 4250F On net _100Mo; 5d': 'd',
                  'FNF2 ( JAPPANTE)': 'o',
                  'SUPERMAGIK_1000': 'o',
                  'DataPack_Incoming': 'o',
                  'MIXT: 500F=75(SMS, ONNET, Mo)_1000FAllNet;24h\t\t': 'h',
                  'MIXT:10000F=10hAllnet_3Go_1h_Zone3;30d\t\t': 'm',
                  'Jokko_Weekly': 'w',
                  'EVC_500=2000F': 'o',
                  'WIFI_ Family _4MBPS': 'o', #WIFI - m?
                  'Data:700F=SPPackage1,7d': 'w',
                  '200F=10mnOnNetValid1H': 'h',
                  'Staff_CPE_Rent': 'o',
                  'FIFA_TS_daily': 'd',
                  'MIXT: 5000F=80Konnet_20Koffnet_250Mo;30d\t\t': 'm',
                  '500=Unlimited3Day': 'd',
                  'IVR Echat_Weekly_200F': 'w',
                  'All-net 500F=4000F ; 5d': 'd',
                  'TelmunCRBT_daily': 'd',
                  'Data: 200F=1GB,24H': 'h',
                  'MIXT: 4900F= 10H on net_1,5Go ;30d': 'm',
                  'VAS(IVR_Radio_Monthly)': 'm',
                  'Mixt : 500F=2500Fonnet_2500Foffnet ;5d': 'd',
                  'Internat: 1000F_Zone_3;24h\t\t': 'h',
                  'Internat: 2000F_Zone_2;24H\t\t': 'h',
                  'Incoming_Bonus_woma': 'o',
                  'EVC_100Mo': 'm', #o
                  'VAS(IVR_Radio_Weekly)': 'w',
                  'CVM_200f=400MB': 'o',
                  'FNF_Youth_ESN': 'o',
                  'WIFI_ Family _10MBPS': 'o',
                  '305155009': 'o',
                  '1000=Unlimited7Day': 'w',
                  'Data_EVC_2Go24H': 'h',
                  'IVR Echat_Monthly_500F': 'm',
                  'NEW_CLIR_PERMANENT_LIBERTE_MOBILE': 'o',
                  'EVC_JOKKO30': 'm',
                  'CVM_100F_unlimited': 'o',
                  'CVM_500f=2GB': 'o',
                  'EVC_Jokko_Weekly': 'w',
                  'CVM_100f=200 MB': 'o',
                  'GPRS_3000Equal10GPORTAL': 'o',
                  'EVC_700Mo': 'm',
                  'CVM_On-net 400f=2200F': 'o',
                  'pilot_offer6': 'o',
                  'All-net 500= 4000off+4000on;24H': 'h',
                  'pack_chinguitel_24h': 'h',
                  'CVM_On-net 1300f=12500': 'o',
                  '150=unlimited pilot auto': 'o',
                  'Data_Mifi_10Go_Monthly': 'm',
                  'EVC_1Go': 'o',
                  'EVC_4900=12000F': 'o',
                  'APANews_weekly': 'w',
                  'EVC_MEGA10000F': 'w',
                  'pilot_offer7': 'o',
                  'NEW_CLIR_TEMPALLOWED_LIBERTE_MOBILE': 'o',
                  'SMS Max': 'o',
                  'MegaChrono_3000F=12500F TOUS RESEAUX': 'o',
                  'NEW_CLIR_TEMPRESTRICTED_LIBERTE_MOBILE': 'o',
                  'GPRS_BKG_1000F MIFI': 'o',
                  'Data:OneTime_Pilot_1.5GB': 'o',
                  'ESN_POSTPAID_CLASSIC_RENT': 'o',
                  'pilot_offer5': 'o',
                  '200=unlimited pilot auto': 'o',
                  'Postpaid FORFAIT 10H Package': 'h',
                  'Data:1000F=700MB,7d': 'w',
                  'CVM_100f=500 onNet': 'o',
                  'FIFA_TS_monthly': 'm',
                  'Go-NetPro-4 Go': 'o',
                  'pilot_offer4': 'o',
                  '1500=Unlimited7Day': 'o',
                  'GPRS_PKG_5GO_ILLIMITE': 'o',
                  'FIFA_TS_weekly': 'w',
                  'Package3_Monthly': 'm',
                  'Data:New-GPRS_PKG_1500F': 'o',
                  'Data_Mifi_10Go': 'o',
                  'GPRS_5Go_7D_PORTAL': 'w',
                  'Data_Mifi_20Go': 'o',
                  'YMGX on-net 100=700F, 24H': 'h',
                  '301765007': 'o',
                  'APANews_monthly': 'm',
                  'CVM_150F_unlimited': 'o'
                  }
    tarif_purp = {'EVC_1000=6000 F' : 'f',
                  'EVC_PACK_2.2Go' : 'f',
                  'All-net 500F=2000F;5d': 'f',
                  'On net 200F=Unlimited _call24H': 'n',
                  'Data:490F=1GB,7d': 'd',
                  'Data: 100 F=40MB,24H': 'd',
                  'Mixt 250F=Unlimited_call24H': 'bu',
                  'MIXT:500F= 2500F on net _2500F off net;2d': 'f',
                  'Data:1000F=2GB,30d': 'd',
                  'All-net 500F =2000F_AllNet_Unlimited': 'o',
                  'Jokko_Daily': 'bu',
                  'Data: 200 F=100MB,24H': 'd',
                  'IVR Echat_Daily_50F': 'o',
                  'On-net 500=4000,10d': 'n',
                  'On-net 500F_FNF;3d': 'n',
                  'Data:200F=Unlimited,24H': 'd',
                  'MIXT: 200mnoff net _unl on net _5Go;30d': 'bu',
                  'On-net 1000F=10MilF;10d': 'n',
                  'VAS(IVR_Radio_Daily)': 'o',
                  'Data:1000F=5GB,7d': 'd',
                  'Twter_U2opia_Daily': 'o',
                  'All-net 600F= 3000F ;5d': 'f',
                  'On-net 200F=60mn;1d': 'n',
                  'All-net 1000=5000;5d': 'f',
                  'Data:500F=2GB,24H': 'd',
                  'Twter_U2opia_Weekly': 'o',
                  'Data:3000F=10GB,30d': 'd',
                  'MIXT: 390F=04HOn-net_400SMS_400 Mo;4h\t': 'bu',
                  'All-net 500F=1250F_AllNet_1250_Onnet;48h': 'bu',
                  'Data:300F=100MB,2d': 'd',
                  'Data:50F=30MB_24H': 'd',
                  'All-net 1000F=(3000F On+3000F Off);5d': 'bu',
                  'On net 200F= 3000F_10Mo ;24H': 'n',
                  'MIXT: 590F=02H_On-net_200SMS_200 Mo;24h\t\t': 'bu',
                  'Data:DailyCycle_Pilot_1.5GB': 'd',
                  'All-net 300=600;2d': 'f',
                  'Data:150F=SPPackage1,24H': 'd',
                  '200=Unlimited1Day': 'bu',
                  'Data:1500F=3GB,30D': 'd',
                  'Jokko_promo': 'bu',
                  'Data:700F=1.5GB,7d': 'd',
                  'On-net 300F=1800F;3d': 'n',
                  'Facebook_MIX_2D': 'o',
                  'Data:30Go_V 30_Days': 'd',
                  'Pilot_Youth4_490': 'd',
                  'CVM_on-net bundle 500=5000': 'n',
                  'MROMO_TIMWES_RENEW': 'bu',
                  'Jokko_Monthly': 'o',
                  'MROMO_TIMWES_OneDAY': 'd',
                  'WIFI_Family_2MBPS': 'd',
                  'Data:1500F=SPPackage1,30d': 'd',
                  'Yewouleen_PKG': 'o',
                  'Twter_U2opia_Monthly': 'o',
                  'YMGX 100=1 hour FNF, 24H/1 month': 'bu',
                  'New_YAKALMA_4_ALL': 'o',
                  'Data: 490F=Night,00H-08H': 'd',
                  'Pilot_Youth1_290': 'd',
                  'Internat: 1000F_Zone_1;24H\t\t': 'bu',
                  'All-net 5000= 20000off+20000on;30d': 'f',
                  'SUPERMAGIK_5000': 'bu',
                  'On-net 2000f_One_Month_100H; 30d': 'n',
                  'MIXT:1000F=4250 Off net _ 4250F On net _100Mo; 5d': 'bu',
                  'FNF2 ( JAPPANTE)': 'n',
                  'SUPERMAGIK_1000': 'bu',
                  'DataPack_Incoming': 'd',
                  'MIXT: 500F=75(SMS, ONNET, Mo)_1000FAllNet;24h\t\t': 'bu',
                  'MIXT:10000F=10hAllnet_3Go_1h_Zone3;30d\t\t': 'bu',
                  'Jokko_Weekly': 'o',
                  'EVC_500=2000F': 'f',
                  'WIFI_ Family _4MBPS': 'd',
                  'Data:700F=SPPackage1,7d': 'd',
                  '200F=10mnOnNetValid1H': 'o',
                  'Staff_CPE_Rent': 'd',
                  'FIFA_TS_daily': 'd',
                  'MIXT: 5000F=80Konnet_20Koffnet_250Mo;30d\t\t': 'bu',
                  '500=Unlimited3Day': 'bu',
                  'IVR Echat_Weekly_200F': 'o',
                  'All-net 500F=4000F ; 5d': 'f',
                  'TelmunCRBT_daily': 'bu',
                  'Data: 200F=1GB,24H': 'd',
                  'MIXT: 4900F= 10H on net_1,5Go ;30d': 'bu',
                  'VAS(IVR_Radio_Monthly)': 'o',
                  'Mixt : 500F=2500Fonnet_2500Foffnet ;5d': 'bu',
                  'Internat: 1000F_Zone_3;24h\t\t': 'bu',
                  'Internat: 2000F_Zone_2;24H\t\t': 'bu',
                  'Incoming_Bonus_woma': 'bu',
                  'EVC_100Mo': 'd',
                  'VAS(IVR_Radio_Weekly)': 'o',
                  'CVM_200f=400MB': 'd',
                  'FNF_Youth_ESN': 'o',
                  'WIFI_ Family _10MBPS': 'd',
                  '305155009': 'd',
                  '1000=Unlimited7Day': 'bu',
                  'Data_EVC_2Go24H': 'd',
                  'IVR Echat_Monthly_500F': 'o',
                  'NEW_CLIR_PERMANENT_LIBERTE_MOBILE': 'o',
                  'EVC_JOKKO30': 'o',
                  'CVM_100F_unlimited': 'n',
                  'CVM_500f=2GB': 'd',
                  'EVC_Jokko_Weekly': 'o',
                  'CVM_100f=200 MB': 'd',
                  'GPRS_3000Equal10GPORTAL': 'd',
                  'EVC_700Mo': 'd',
                  'CVM_On-net 400f=2200F': 'n',
                  'pilot_offer6': 'o',
                  'All-net 500= 4000off+4000on;24H': 'bu',
                  'pack_chinguitel_24h': 'o',
                  'CVM_On-net 1300f=12500': 'n',
                  '150=unlimited pilot auto': 'o',
                  'Data_Mifi_10Go_Monthly': 'd',
                  'EVC_1Go': 'o',
                  'EVC_4900=12000F': 'bu',
                  'APANews_weekly': 'd',
                  'EVC_MEGA10000F': 'n',
                  'pilot_offer7': 'o',
                  'NEW_CLIR_TEMPALLOWED_LIBERTE_MOBILE': 'f',
                  'SMS Max': 'o',
                  'MegaChrono_3000F=12500F TOUS RESEAUX': 'bu',
                  'NEW_CLIR_TEMPRESTRICTED_LIBERTE_MOBILE': 'f',
                  'GPRS_BKG_1000F MIFI': 'd',
                  'Data:OneTime_Pilot_1.5GB': 'd',
                  'ESN_POSTPAID_CLASSIC_RENT': 'f',
                  'pilot_offer5': 'o',
                  '200=unlimited pilot auto': 'o',
                  'Postpaid FORFAIT 10H Package': 'bu',
                  'Data:1000F=700MB,7d': 'd',
                  'CVM_100f=500 onNet': 'n',
                  'FIFA_TS_monthly': 'd',
                  'Go-NetPro-4 Go': 'bu',
                  'pilot_offer4': 'd',
                  '1500=Unlimited7Day': 'bu',
                  'GPRS_PKG_5GO_ILLIMITE': 'd',
                  'FIFA_TS_weekly': 'd',
                  'Package3_Monthly': 'd',
                  'Data:New-GPRS_PKG_1500F': 'd',
                  'Data_Mifi_10Go': 'd',
                  'GPRS_5Go_7D_PORTAL': 'd',
                  'Data_Mifi_20Go': 'd',
                  'YMGX on-net 100=700F, 24H': 'n',
                  '301765007': 'f',
                  'APANews_monthly': 'o',
                  'CVM_150F_unlimited': 'bu'
                  }
    tarif_lims = {'EVC_1000=6000 F' : 'l',
                  'EVC_PACK_2.2Go' : 'l',
                  'All-net 500F=2000F;5d': 'l',
                  'On net 200F=Unlimited _call24H': 'u',
                  'Data:490F=1GB,7d': 'l',
                  'Data: 100 F=40MB,24H': 'l',
                  'Mixt 250F=Unlimited_call24H': 'u',
                  'MIXT:500F= 2500F on net _2500F off net;2d': 'l',
                  'Data:1000F=2GB,30d': 'l',
                  'All-net 500F =2000F_AllNet_Unlimited': 'u',
                  'Jokko_Daily': 'o',
                  'Data: 200 F=100MB,24H': 'l',
                  'IVR Echat_Daily_50F': 'o', #l?
                  'On-net 500=4000,10d': 'l',
                  'On-net 500F_FNF;3d': 'l',
                  'Data:200F=Unlimited,24H': 'u',
                  'MIXT: 200mnoff net _unl on net _5Go;30d': 'l',
                  'On-net 1000F=10MilF;10d': 'l',
                  'VAS(IVR_Radio_Daily)': 'o',
                  'Data:1000F=5GB,7d': 'l',
                  'Twter_U2opia_Daily': 'o', #u?
                  'All-net 600F= 3000F ;5d': 'l',
                  'On-net 200F=60mn;1d': 'l',
                  'All-net 1000=5000;5d': 'l',
                  'Data:500F=2GB,24H': 'l',
                  'Twter_U2opia_Weekly': 'o',
                  'Data:3000F=10GB,30d': 'l',
                  'MIXT: 390F=04HOn-net_400SMS_400 Mo;4h\t': 'l',
                  'All-net 500F=1250F_AllNet_1250_Onnet;48h': 'l',
                  'Data:300F=100MB,2d': 'l',
                  'Data:50F=30MB_24H': 'l',
                  'All-net 1000F=(3000F On+3000F Off);5d': 'l',
                  'On net 200F= 3000F_10Mo ;24H': 'l',
                  'MIXT: 590F=02H_On-net_200SMS_200 Mo;24h\t\t': 'l',
                  'Data:DailyCycle_Pilot_1.5GB': 'l',
                  'All-net 300=600;2d': 'l',
                  'Data:150F=SPPackage1,24H': 'l',
                  '200=Unlimited1Day': 'u',
                  'Data:1500F=3GB,30D': 'l',
                  'Jokko_promo': 'o',
                  'Data:700F=1.5GB,7d': 'l',
                  'On-net 300F=1800F;3d': 'l',
                  'Facebook_MIX_2D': 'o', #l
                  'Data:30Go_V 30_Days': 'l',
                  'Pilot_Youth4_490': 'o',
                  'CVM_on-net bundle 500=5000': 'l',
                  'MROMO_TIMWES_RENEW': 'o',
                  'Jokko_Monthly': 'o',
                  'MROMO_TIMWES_OneDAY': 'o',
                  'WIFI_Family_2MBPS': 'o',
                  'Data:1500F=SPPackage1,30d': 'l',
                  'Yewouleen_PKG': 'o',
                  'Twter_U2opia_Monthly': 'o',
                  'YMGX 100=1 hour FNF, 24H/1 month': 'o',
                  'New_YAKALMA_4_ALL': 'o',
                  'Data: 490F=Night,00H-08H': 'l',
                  'Pilot_Youth1_290': 'o',
                  'Internat: 1000F_Zone_1;24H\t\t': 'o',
                  'All-net 5000= 20000off+20000on;30d': 'l',
                  'SUPERMAGIK_5000': 'o',
                  'On-net 2000f_One_Month_100H; 30d': 'l',
                  'MIXT:1000F=4250 Off net _ 4250F On net _100Mo; 5d': 'l',
                  'FNF2 ( JAPPANTE)': 'o',
                  'SUPERMAGIK_1000': 'o',
                  'DataPack_Incoming': 'l',
                  'MIXT: 500F=75(SMS, ONNET, Mo)_1000FAllNet;24h\t\t': 'l',
                  'MIXT:10000F=10hAllnet_3Go_1h_Zone3;30d\t\t': 'l',
                  'Jokko_Weekly': 'o',
                  'EVC_500=2000F': 'l',
                  'WIFI_ Family _4MBPS': 'o',
                  'Data:700F=SPPackage1,7d': 'l',
                  '200F=10mnOnNetValid1H': 'l',
                  'Staff_CPE_Rent': 'o',
                  'FIFA_TS_daily': 'o',
                  'MIXT: 5000F=80Konnet_20Koffnet_250Mo;30d\t\t': 'l',
                  '500=Unlimited3Day': 'u',
                  'IVR Echat_Weekly_200F': 'o',
                  'All-net 500F=4000F ; 5d': 'l',
                  'TelmunCRBT_daily': 'o',
                  'Data: 200F=1GB,24H': 'l',
                  'MIXT: 4900F= 10H on net_1,5Go ;30d': 'l',
                  'VAS(IVR_Radio_Monthly)': 'o',
                  'Mixt : 500F=2500Fonnet_2500Foffnet ;5d': 'l',
                  'Internat: 1000F_Zone_3;24h\t\t': 'o',
                  'Internat: 2000F_Zone_2;24H\t\t': 'o',
                  'Incoming_Bonus_woma': 'o',
                  'EVC_100Mo': 'l',
                  'VAS(IVR_Radio_Weekly)': 'o',
                  'CVM_200f=400MB': 'l',
                  'FNF_Youth_ESN': 'o',
                  'WIFI_ Family _10MBPS': 'o',
                  '305155009': 'o',
                  '1000=Unlimited7Day': 'u',
                  'Data_EVC_2Go24H': 'l',
                  'IVR Echat_Monthly_500F': 'o',
                  'NEW_CLIR_PERMANENT_LIBERTE_MOBILE': 'o',
                  'EVC_JOKKO30': 'o',
                  'CVM_100F_unlimited': 'u',
                  'CVM_500f=2GB': 'l',
                  'EVC_Jokko_Weekly': 'o',
                  'CVM_100f=200 MB': 'l',
                  'GPRS_3000Equal10GPORTAL': 'l',
                  'EVC_700Mo': 'l',
                  'CVM_On-net 400f=2200F': 'l',
                  'pilot_offer6': 'o',
                  'All-net 500= 4000off+4000on;24H': 'l',
                  'pack_chinguitel_24h': 'o',
                  'CVM_On-net 1300f=12500': 'l',
                  '150=unlimited pilot auto': 'o',
                  'Data_Mifi_10Go_Monthly': 'o',
                  'EVC_1Go' : 'o',
                  'EVC_4900=12000F': 'l',
                  'APANews_weekly': 'o',
                  'EVC_MEGA10000F': 'l',
                  'pilot_offer7': 'o',
                  'NEW_CLIR_TEMPALLOWED_LIBERTE_MOBILE': 'o',
                  'SMS Max': 'u',
                  'MegaChrono_3000F=12500F TOUS RESEAUX': 'l',
                  'NEW_CLIR_TEMPRESTRICTED_LIBERTE_MOBILE': 'o',
                  'GPRS_BKG_1000F MIFI': 'l',
                  'Data:OneTime_Pilot_1.5GB': 'l',
                  'ESN_POSTPAID_CLASSIC_RENT': 'l',
                  'pilot_offer5': 'o',
                  '200=unlimited pilot auto': 'u',
                  'Postpaid FORFAIT 10H Package': 'l',
                  'Data:1000F=700MB,7d': 'l',
                  'CVM_100f=500 onNet': 'l',
                  'FIFA_TS_monthly': 'l',
                  'Go-NetPro-4 Go': 'l',
                  'pilot_offer4': 'o',
                  '1500=Unlimited7Day': 'u',
                  'GPRS_PKG_5GO_ILLIMITE': 'l',
                  'FIFA_TS_weekly': 'o',
                  'Package3_Monthly': 'l',
                  'Data:New-GPRS_PKG_1500F': 'l',
                  'Data_Mifi_10Go': 'l',
                  'GPRS_5Go_7D_PORTAL': 'o',
                  'Data_Mifi_20Go': 'l',
                  'YMGX on-net 100=700F, 24H': 'l',
                  '301765007': 'o',
                  'APANews_monthly': 'u',
                  'CVM_150F_unlimited': 'u'
                  }
    #hourly (h); dayly (d); weekly (w); monthly (m); other (o)
    # on_net first (n); universal: calls first (f)/ data first (d); balanced/unclear (bu); other (o)
    #limited (l); unlimited (u); other (o)
    
    train_y.insert(train_y.shape[1], 'T_TIME', train_y['TOP_PACK'].replace(to_replace=tarif_time, inplace=False))
    train_y['T_TIME'].fillna('n', inplace = True)
    
    train_y.insert(train_y.shape[1], 'T_PURP', train_y['TOP_PACK'].replace(to_replace=tarif_purp, inplace=False))
    train_y['T_PURP'].fillna('n', inplace = True)
    
    train_y.insert(train_y.shape[1], 'T_LIMS', train_y['TOP_PACK'].replace(to_replace=tarif_lims, inplace=False))
    train_y['T_LIMS'].fillna('n', inplace = True)
    
    
    
    #Finish label encoding
    
    time_cats = {'h' : 0, 'n' : 1, 'w' : 2, 'd' : 3, 'm' : 4, 'o' : 5}
    purp_cats = {'n' : 0, 'd' : 1, 'bu' : 2, 'f' : 3, 'o' : 4}
    lims_cats = {'u' : 0, 'n' : 1, 'l' : 2, 'o' : 3}
    train_y['T_TIME'].replace(to_replace=time_cats, inplace=True)
    train_y['T_PURP'].replace(to_replace=purp_cats, inplace=True)
    train_y['T_LIMS'].replace(to_replace=lims_cats, inplace=True)
    
    
    
    # Regional median (and mean) deviation

    train_y['REG_MONTANT_MEDIAN']=train_y.groupby(['REGION'])['MONTANT'].apply(lambda x: x-x.median())
    train_y['REG_MONTANT_MEDIAN'].fillna(0, inplace=True)
    train_y['REG_FREQUENCE_RECH_MEDIAN']=train_y.groupby(['REGION'])['FREQUENCE_RECH'].apply(lambda x: x-x.median())
    train_y['REG_FREQUENCE_RECH_MEDIAN'].fillna(0, inplace=True)
    train_y['REG_REVENUE_MEDIAN']=train_y.groupby(['REGION'])['REVENUE'].apply(lambda x: x-x.median())
    train_y['REG_REVENUE_MEDIAN'].fillna(0, inplace=True)
    train_y['REG_FREQUENCE_MEDIAN']=train_y.groupby(['REGION'])['FREQUENCE'].apply(lambda x: x-x.median())
    train_y['REG_FREQUENCE_MEDIAN'].fillna(0, inplace=True)
    train_y['REG_REGULARITY_MEDIAN']=train_y.groupby(['REGION'])['REGULARITY'].apply(lambda x: x-x.median())
    train_y['REG_FREQ_TOP_PACK_MEDIAN']=train_y.groupby(['REGION'])['FREQ_TOP_PACK'].apply(lambda x: x-x.median())
    train_y['REG_FREQ_TOP_PACK_MEDIAN'].fillna(0, inplace=True)
    
    train_y['REG_ON_NET_MEDIAN']=train_y.groupby(['REGION'])['ON_NET'].apply(lambda x: x-x.median())
    train_y['REG_ON_NET_MEDIAN'].fillna(-1, inplace=True)
    train_y['REG_ON_NET_MEAN']=train_y.groupby(['REGION'])['ON_NET'].apply(lambda x: x-x.mean())
    train_y['REG_ON_NET_MEAN'].fillna(-1, inplace=True)
    
    train_y['REG_ORANGE_MEDIAN']=train_y.groupby(['REGION'])['ORANGE'].apply(lambda x: x-x.median())
    train_y['REG_ORANGE_MEDIAN'].fillna(-1, inplace=True)
    train_y['REG_ORANGE_MEAN']=train_y.groupby(['REGION'])['ORANGE'].apply(lambda x: x-x.mean())
    train_y['REG_ORANGE_MEAN'].fillna(-1, inplace=True)
    
    train_y['REG_DATA_VOLUME_MEDIAN']=train_y.groupby(['REGION'])['DATA_VOLUME'].apply(lambda x: x-x.median())
    train_y['REG_DATA_VOLUME_MEDIAN'].fillna(-1, inplace=True)
    train_y['REG_DATA_VOLUME_MEAN']=train_y.groupby(['REGION'])['DATA_VOLUME'].apply(lambda x: x-x.mean())
    train_y['REG_DATA_VOLUME_MEAN'].fillna(-1, inplace=True)
    
    train_y['REG_FULL_CALLS_SUM_MEDIAN']=train_y.groupby(['REGION'])['FULL_CALLS_SUM'].apply(lambda x: x-x.median())
    train_y['REG_FULL_CALLS_SUM_MEDIAN'].fillna(-1, inplace=True)        
    train_y['REG_FULL_CALLS_SUM_MEAN']=train_y.groupby(['REGION'])['FULL_CALLS_SUM'].apply(lambda x: x-x.mean())
    train_y['REG_FULL_CALLS_SUM_MEAN'].fillna(-1, inplace=True)
    
    train_y['FREQ_TOP_PACK'].fillna(0, inplace = True)
    train_y['MONTANT'].fillna(0, inplace = True)
    train_y['FREQUENCE_RECH'].fillna(0, inplace = True)
    train_y['REVENUE'].fillna(0, inplace = True)
    train_y['FREQUENCE'].fillna(0, inplace = True)
    train_y['DATA_VOLUME'].replace(to_replace=0, value=1, inplace=True)
    train_y['ON_NET'].replace(to_replace=0, value=1, inplace=True)
    train_y['ORANGE'].replace(to_replace=0, value=1, inplace=True)
    train_y['TIGO'].replace(to_replace=0, value=1, inplace=True)
    train_y['ZONE1'].replace(to_replace=0, value=1, inplace=True)
    train_y['ZONE2'].replace(to_replace=0, value=1, inplace=True)
    train_y.fillna(0, inplace=True)
    
    #Popularity general
    
    train_y["POPULARITY"]=train_y["TOP_PACK"].map(train_y[train_y["TOP_PACK"].notnull()]["TOP_PACK"].value_counts()/len(train_y[train_y["TOP_PACK"].notnull()]["TOP_PACK"]))
    
    train_y.drop('TOP_PACK', axis=1, inplace=True)
    # Move target to last position
    if target is not None:
        train_y.insert(train_y.shape[1], 'CHURN', target)

    return(train_y)

In [4]:
Preproc_train = data_preparation(train)
Preproc_train.to_csv('dataset_5_train.csv')

In [5]:
Preproc_test = data_preparation(test)
Preproc_test.to_csv('dataset_5_test.csv')

In [6]:
train_merged = data_preparation(merged)
dirty_train = train_merged.iloc[:train.shape[0]]
dirty_test = train_merged[~train_merged.index.isin(dirty_train.index)]
dirty_train.to_csv('dataset_5_train_merged.csv')
dirty_test.to_csv('dataset_5_test_merged.csv')